In [1]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os
import joblib
from discussion_agents.cog.agent.expel import ExpeLAgent

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)
hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')


c:\Users\tuvin\anaconda3\envs\discussion-agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm
)

In [3]:
agent.gather_experience(
    questions=hotpotqa.question.values[:1],
    keys=hotpotqa.answer.values[:1]
)

In [ ]:
# examples = self.experience_memory.load_memories(
#     query=question,
#     k_docs=k_docs, 
#     num_fewshots=num_fewshots, 
#     max_fewshot_tokens=max_fewshot_tokens,
#     reranker_strategy=reranker_strategy
# )['fewshots']
# examples = examples if examples else [REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES]
# examples = "\n\n".join(examples + [END_OF_EXAMPLES_DELIMITER]) + "\n"

# # Dynamically load in all insights.
# examples += RULE_PREFIX
# insights = self.insight_memory.load_memories()['insights']
# insights = "".join([f"{i}. {insight['insight']}\n" for i, insight in enumerate(insights, 1)])
# examples += insights

In [18]:
import joblib 
from discussion_agents.cog.modules.memory.expel import ExpeLExperienceMemory, ExpeLInsightMemory

hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')
experiences = joblib.load('../../../tests/assets/expel/expel_experiences_10_fake.joblib')
memory = ExpeLExperienceMemory(experiences)

In [19]:
queries = {
    "task": 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
    "thought": 'Thought: I should try a different approach. Let me search for press releases, industry news sources, or announcements specifically related to the name change and new acronym for VIVA Media AG in 2004. By focusing on more specialized sources, I may be able to find the accurate information needed to answer the question correctly. ',
    "other": "Some other query."
}
RULE_PREFIX = """
The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:
"""

memory_dict = memory.load_memories(query=queries["task"])['fewshots']

In [20]:
insights = [
    {"insight": "Insight 1", "score": 1},
    {"insight": "Insight 2", "score": 2},
    {"insight": "Insight 3", "score": 3}
]
memory = ExpeLInsightMemory(insights=insights, max_num_insights=5, leeway=1)

insights = memory.load_memories()['insights']

c = "".join([f"{i}. {insight['insight']}\n" for i, insight in enumerate(insights, 1)])

In [23]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES

In [24]:
a = "\n\n".join([REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES] + ["(END OF EXAMPLES)"]) + "\n"
a += RULE_PREFIX
a += c + "\n"
a += "What is deep learning?"
print(a)

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions:
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[H

In [ ]:
EXPEL_REFLEXION_REACT_INSTRUCTION = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of {max_steps} steps.

Here are some examples:
{examples}

{reflections}

Question: {question}{scratchpad}
"""

examples = "\n\n".join(memory_dict) + "\n\n" + "(END OF EXAMPLES)" + "\n\n"
print(examples)

In [ ]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm
)

In [ ]:
agent.insight_memory.insights

In [ ]:
agent.gather_experience(
    questions=hotpotqa.question.values[:1],
    keys=hotpotqa.answer.values[:1]
)